<a href="https://colab.research.google.com/github/deokwoo-han/abc/blob/master/10_07_%EC%97%B0%EA%B4%80%EB%B6%84%EC%84%9D(dept).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
color = sns.color_palette()

%matplotlib inline
%config InlineBackend.figure_config = 'retina'

pd.options.mode.chained_assignment = None  # default='warn'

###1. 구글 드라이버 파일 가져오기(시행자 경로에 해당 파일이 있어야 함)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pwd cd

/content


###2. 파일 불러와서 데이터 프레임으로 만들기

In [4]:
file_path = './drive/MyDrive/datas'
order_products_train = pd.read_csv(file_path + "/order_products__train.csv")
order_products_prior = pd.read_csv(file_path + "/order_products__prior.csv")
orders = pd.read_csv(file_path + "/orders.csv")
products = pd.read_csv(file_path + "/products.csv")
aisles = pd.read_csv(file_path + "/aisles.csv")
departments = pd.read_csv(file_path + "/departments.csv")

###3. 데이터 프레임 통합

In [5]:
order_products_prior = pd.merge(order_products_prior, products, on='product_id', how='left')
order_products_prior = pd.merge(order_products_prior, aisles, on='aisle_id', how='left')
order_products_prior = pd.merge(order_products_prior, departments, on='department_id', how='left')
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry


###4. order_id와 department(제품 부서, 제품군)만 데이터프레임화

In [6]:
df2 = pd.DataFrame(order_products_prior.groupby('order_id').department.agg(lambda x: x.tolist())).reset_index()
df2.head(10)

,order_id,department
0,2,"[dairy eggs, produce, pantry, pantry, pantry, ..."
1,3,"[dairy eggs, dairy eggs, produce, produce, dai..."
2,4,"[bakery, personal care, snacks, breakfast, bre..."
3,5,"[produce, pantry, deli, produce, dairy eggs, h..."
4,6,"[beverages, household, household]"
5,7,"[beverages, frozen]"
6,8,[bakery]
7,9,"[produce, produce, produce, dairy eggs, snacks..."
8,10,"[produce, produce, produce, produce, produce, ..."
9,11,"[frozen, deli, canned goods, snacks, pantry]"


In [7]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

###5. 각 주문별 제품군 구매 여부

In [8]:
te = TransactionEncoder() 
te_result = te.fit(df2.department).transform(df2.department)
df2_1 = pd.DataFrame( te_result, columns = te.columns_)
df2_1.head()

,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,frozen,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False
1,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
3,False,False,False,True,False,False,False,True,True,True,False,True,True,True,False,False,True,True,False,True,True
4,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


###6. 제품군 구매 사례별 지지도

In [9]:
itemset1 = apriori(df2_1, use_colnames=True, min_support=0.03) 

In [10]:
itemset1

,support,itemsets
0,0.055278,(babies)
1,0.274212,(bakery)
2,0.453315,(beverages)
3,0.163362,(breakfast)
4,0.211923,(canned goods)
...,...,...
718,0.031293,"(frozen, dry goods pasta, snacks, beverages, d..."
719,0.045762,"(frozen, snacks, pantry, beverages, dairy eggs..."
720,0.032411,"(frozen, canned goods, snacks, pantry, dairy e..."
721,0.034251,"(frozen, deli, pantry, snacks, dairy eggs, pro..."


### 7. 각 제품군 간의 연관 규칙 분석(0.03 이상의 지지도가 나온 규칙들 lift 순)

In [11]:
df2_2 = association_rules(itemset1, metric="confidence", min_threshold=0.85 )
df2_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(canned goods),(produce),0.211923,0.749429,0.186306,0.879123,1.173057,0.027485,2.072944
1,(deli),(produce),0.239605,0.749429,0.206059,0.859995,1.147533,0.026492,1.789726
2,(dry goods pasta),(produce),0.185967,0.749429,0.161743,0.869737,1.160533,0.022373,1.923585
3,(international),(produce),0.068910,0.749429,0.059938,0.869805,1.160624,0.008295,1.924583
4,(meat seafood),(produce),0.178772,0.749429,0.156475,0.875276,1.167923,0.022498,2.008996
...,...,...,...,...,...,...,...,...,...
561,"(frozen, deli, pantry, snacks, produce)",(dairy eggs),0.037161,0.677270,0.034251,0.921695,1.360897,0.009083,4.121421
562,"(deli, frozen, pantry, snacks)","(dairy eggs, produce)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
563,"(frozen, dry goods pasta, pantry, snacks, dair...",(produce),0.033143,0.749429,0.030741,0.927545,1.237669,0.005903,3.458301
564,"(frozen, dry goods pasta, pantry, snacks, prod...",(dairy eggs),0.033124,0.677270,0.030741,0.928076,1.370319,0.008308,4.487121


antecedents: 비교할 선행 제품군들

consequents: 비교할 후행 제품군들

antecedent support: 전체 주문 대비 선행 제품군들 구매비율(지지도)

consequent support: 전체 주문 대비 후행 제품군들 구매비율(지지도)

support: 두 제품군 동시 구매 비율(지지도)

confidence: 선행 제품군이 있을 때 후행 제품군도 있는 비율(조건부 확률, 신뢰도)

lift: 동시 구매 여부를 알기 위해 서로 간의 연관성을 파악하는 향상도(1보다 높을 경우 itemA, itemB 두 품목 간의 긍정적 관계, 낮을 경우 두 품목 간의 부정적 관계, 1일 경우 관계 없음)

leverage: 0에 가깝다면 각각 독립일 가능성이 높음. 양수면 향상도가 1보다 큰 경우와 유사, 음수면 향상도가 1보다 작은 경우와 유사

conviction: 독립 확률 지표. 1보다 크면 상호연관성 높음. 1보다 작으면 상호 연관성 낮음. 1이면 상호연관성 없음(독립)



In [12]:
rules_final = df2_2.sort_values('support', ascending=False)
rules_final.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
72,"(dairy eggs, frozen)",(produce),0.283380,0.749429,0.242580,0.856024,1.142236,0.030207,1.740372
77,"(dairy eggs, pantry)",(produce),0.269262,0.749429,0.232459,0.863319,1.151969,0.030666,1.833251
1,(deli),(produce),0.239605,0.749429,0.206059,0.859995,1.147533,0.026492,1.789726
18,"(bakery, dairy eggs)",(produce),0.224997,0.749429,0.194596,0.864883,1.154056,0.025977,1.854479
0,(canned goods),(produce),0.211923,0.749429,0.186306,0.879123,1.173057,0.027485,2.072944
64,"(dairy eggs, deli)",(produce),0.193060,0.749429,0.171777,0.889760,1.187251,0.027092,2.272962
2,(dry goods pasta),(produce),0.185967,0.749429,0.161743,0.869737,1.160533,0.022373,1.923585
4,(meat seafood),(produce),0.178772,0.749429,0.156475,0.875276,1.167923,0.022498,2.008996
51,"(canned goods, dairy eggs)",(produce),0.169985,0.749429,0.153674,0.904046,1.206313,0.026283,2.611362
319,"(frozen, snacks, produce)",(dairy eggs),0.163498,0.677270,0.140050,0.856585,1.264761,0.029318,2.250316


In [13]:
rules_final.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
72,"(dairy eggs, frozen)",(produce),0.283380,0.749429,0.242580,0.856024,1.142236,0.030207,1.740372
77,"(dairy eggs, pantry)",(produce),0.269262,0.749429,0.232459,0.863319,1.151969,0.030666,1.833251
1,(deli),(produce),0.239605,0.749429,0.206059,0.859995,1.147533,0.026492,1.789726
18,"(bakery, dairy eggs)",(produce),0.224997,0.749429,0.194596,0.864883,1.154056,0.025977,1.854479
0,(canned goods),(produce),0.211923,0.749429,0.186306,0.879123,1.173057,0.027485,2.072944
64,"(dairy eggs, deli)",(produce),0.193060,0.749429,0.171777,0.889760,1.187251,0.027092,2.272962
2,(dry goods pasta),(produce),0.185967,0.749429,0.161743,0.869737,1.160533,0.022373,1.923585
4,(meat seafood),(produce),0.178772,0.749429,0.156475,0.875276,1.167923,0.022498,2.008996
51,"(canned goods, dairy eggs)",(produce),0.169985,0.749429,0.153674,0.904046,1.206313,0.026283,2.611362
319,"(frozen, snacks, produce)",(dairy eggs),0.163498,0.677270,0.140050,0.856585,1.264761,0.029318,2.250316


In [22]:
final = rules_final.reset_index(drop=True)

In [40]:
f= final[:100]

In [41]:
f.tail(40)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
60,"(bakery, deli)",(dairy eggs),0.097857,0.677270,0.086211,0.880988,1.300792,0.019935,2.711738
61,"(snacks, pantry, beverages)",(produce),0.100074,0.749429,0.085464,0.854008,1.139545,0.010466,1.716338
62,"(canned goods, snacks, produce)",(dairy eggs),0.097815,0.677270,0.085360,0.872663,1.288500,0.019112,2.534449
63,"(canned goods, dairy eggs, snacks)",(produce),0.093563,0.749429,0.085360,0.912328,1.217364,0.015241,2.858054
64,"(snacks, pantry, beverages)",(dairy eggs),0.100074,0.677270,0.085204,0.851410,1.257120,0.017427,2.171948
65,"(breakfast, snacks)",(dairy eggs),0.099224,0.677270,0.084888,0.855518,1.263185,0.017687,2.233698
66,"(canned goods, pantry, produce)",(dairy eggs),0.098711,0.677270,0.084831,0.859388,1.268900,0.017977,2.295185
67,"(canned goods, dairy eggs, pantry)",(produce),0.092028,0.749429,0.084831,0.921794,1.229994,0.015862,3.203968
68,"(frozen, pantry, snacks)",(produce),0.094783,0.749429,0.083344,0.879311,1.173307,0.012311,2.076163
69,"(dry goods pasta, pantry)",(produce),0.092687,0.749429,0.083256,0.898247,1.198575,0.013793,2.462537


In [14]:
rules_final1 = df2_2.sort_values('lift', ascending=False)
rules_final1.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
503,"(deli, dry goods pasta, pantry)","(dairy eggs, produce)",0.035375,0.550483,0.030457,0.860965,1.564017,0.010983,3.233125
506,"(deli, dry goods pasta, snacks)","(dairy eggs, produce)",0.041999,0.550483,0.035958,0.856172,1.555308,0.012839,3.125367
543,"(deli, bakery, frozen, snacks)","(dairy eggs, produce)",0.038470,0.550483,0.032851,0.853942,1.551258,0.011674,3.077660
500,"(deli, dry goods pasta, frozen)","(dairy eggs, produce)",0.039724,0.550483,0.033869,0.852617,1.548851,0.012002,3.049987
481,"(canned goods, deli, pantry)","(dairy eggs, produce)",0.039601,0.550483,0.033757,0.852432,1.548516,0.011957,3.046170
565,"(dry goods pasta, frozen, pantry, snacks)","(dairy eggs, produce)",0.036083,0.550483,0.030741,0.851967,1.547670,0.010878,3.036593
562,"(deli, frozen, pantry, snacks)","(dairy eggs, produce)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
368,"(produce, breakfast, frozen, bakery)",(dairy eggs),0.033578,0.677270,0.031396,0.935016,1.380566,0.008655,4.966314
370,"(produce, breakfast, snacks, bakery)",(dairy eggs),0.039241,0.677270,0.036592,0.932504,1.376856,0.010016,4.781444
542,"(frozen, bakery, deli, snacks, produce)",(dairy eggs),0.035234,0.677270,0.032851,0.932376,1.376668,0.008988,4.772400


In [15]:
rules_final1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
503,"(deli, dry goods pasta, pantry)","(dairy eggs, produce)",0.035375,0.550483,0.030457,0.860965,1.564017,0.010983,3.233125
506,"(deli, dry goods pasta, snacks)","(dairy eggs, produce)",0.041999,0.550483,0.035958,0.856172,1.555308,0.012839,3.125367
543,"(deli, bakery, frozen, snacks)","(dairy eggs, produce)",0.038470,0.550483,0.032851,0.853942,1.551258,0.011674,3.077660
500,"(deli, dry goods pasta, frozen)","(dairy eggs, produce)",0.039724,0.550483,0.033869,0.852617,1.548851,0.012002,3.049987
481,"(canned goods, deli, pantry)","(dairy eggs, produce)",0.039601,0.550483,0.033757,0.852432,1.548516,0.011957,3.046170
...,...,...,...,...,...,...,...,...,...
72,"(dairy eggs, frozen)",(produce),0.283380,0.749429,0.242580,0.856024,1.142236,0.030207,1.740372
203,"(dairy eggs, frozen, beverages)",(produce),0.149871,0.749429,0.128228,0.855594,1.141661,0.015911,1.735183
44,"(breakfast, frozen)",(produce),0.079575,0.749429,0.068064,0.855342,1.141325,0.008428,1.732160
221,"(snacks, pantry, beverages)",(produce),0.100074,0.749429,0.085464,0.854008,1.139545,0.010466,1.716338


In [46]:
finale = rules_final1[:100].reset_index(drop = True)

In [47]:
finale

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(deli, dry goods pasta, pantry)","(dairy eggs, produce)",0.035375,0.550483,0.030457,0.860965,1.564017,0.010983,3.233125
1,"(deli, dry goods pasta, snacks)","(dairy eggs, produce)",0.041999,0.550483,0.035958,0.856172,1.555308,0.012839,3.125367
2,"(deli, bakery, frozen, snacks)","(dairy eggs, produce)",0.038470,0.550483,0.032851,0.853942,1.551258,0.011674,3.077660
3,"(deli, dry goods pasta, frozen)","(dairy eggs, produce)",0.039724,0.550483,0.033869,0.852617,1.548851,0.012002,3.049987
4,"(canned goods, deli, pantry)","(dairy eggs, produce)",0.039601,0.550483,0.033757,0.852432,1.548516,0.011957,3.046170
...,...,...,...,...,...,...,...,...,...
95,"(canned goods, frozen, bakery)",(dairy eggs),0.046469,0.677270,0.042231,0.908784,1.341834,0.010758,3.538093
96,"(dry goods pasta, frozen, pantry, produce)",(dairy eggs),0.048973,0.677270,0.044488,0.908423,1.341301,0.011320,3.524131
97,"(bakery, dry goods pasta, produce)",(dairy eggs),0.068106,0.677270,0.061866,0.908382,1.341240,0.015740,3.522550
98,"(bakery, deli, pantry)",(dairy eggs),0.049598,0.677270,0.045048,0.908266,1.341069,0.011457,3.518102


In [48]:
finale.head(60)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(deli, dry goods pasta, pantry)","(dairy eggs, produce)",0.035375,0.550483,0.030457,0.860965,1.564017,0.010983,3.233125
1,"(deli, dry goods pasta, snacks)","(dairy eggs, produce)",0.041999,0.550483,0.035958,0.856172,1.555308,0.012839,3.125367
2,"(deli, bakery, frozen, snacks)","(dairy eggs, produce)",0.038470,0.550483,0.032851,0.853942,1.551258,0.011674,3.077660
3,"(deli, dry goods pasta, frozen)","(dairy eggs, produce)",0.039724,0.550483,0.033869,0.852617,1.548851,0.012002,3.049987
4,"(canned goods, deli, pantry)","(dairy eggs, produce)",0.039601,0.550483,0.033757,0.852432,1.548516,0.011957,3.046170
5,"(dry goods pasta, frozen, pantry, snacks)","(dairy eggs, produce)",0.036083,0.550483,0.030741,0.851967,1.547670,0.010878,3.036593
6,"(deli, frozen, pantry, snacks)","(dairy eggs, produce)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
7,"(produce, breakfast, frozen, bakery)",(dairy eggs),0.033578,0.677270,0.031396,0.935016,1.380566,0.008655,4.966314
8,"(produce, breakfast, snacks, bakery)",(dairy eggs),0.039241,0.677270,0.036592,0.932504,1.376856,0.010016,4.781444
9,"(frozen, bakery, deli, snacks, produce)",(dairy eggs),0.035234,0.677270,0.032851,0.932376,1.376668,0.008988,4.772400


In [49]:
finale.tail(40)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
60,"(meat seafood, bakery, produce, beverages)",(dairy eggs),0.034568,0.67727,0.031658,0.915804,1.352198,0.008246,3.833060
61,"(breakfast, frozen, produce, beverages)",(dairy eggs),0.039812,0.67727,0.036459,0.915790,1.352178,0.009496,3.832425
62,"(bakery, dry goods pasta, frozen)",(dairy eggs),0.046179,0.67727,0.042290,0.915788,1.352176,0.011014,3.832360
63,"(bakery, deli, produce, beverages)",(dairy eggs),0.049040,0.67727,0.044880,0.915171,1.351265,0.011667,3.804492
64,"(meat seafood, breakfast, produce)",(dairy eggs),0.032867,0.67727,0.030071,0.914928,1.350906,0.007811,3.793629
65,"(produce, canned goods, bakery, beverages)",(dairy eggs),0.039978,0.67727,0.036577,0.914919,1.350893,0.009501,3.793220
66,"(deli, pantry, snacks, beverages, produce)",(dairy eggs),0.036601,0.67727,0.033484,0.914845,1.350783,0.008696,3.789918
67,"(meat seafood, deli, snacks, produce)",(dairy eggs),0.034037,0.67727,0.031113,0.914096,1.349677,0.008061,3.756851
68,"(produce, canned goods, pantry, bakery)",(dairy eggs),0.042725,0.67727,0.039050,0.913969,1.349489,0.010113,3.751308
69,"(beverages, dry goods pasta, pantry, snacks)",(dairy eggs),0.033270,0.67727,0.030401,0.913761,1.349183,0.007868,3.742288


### 8. 데이터프레임 CSV화

In [16]:
rules_final.to_csv('sample2.csv')

*lift로 살펴 봤을 때, dairy eggs와 produce 제품군에 속한 것을 다른 제품군과 같이 구매하는 경우가 많았다. 이를 볼 때, 재구매활동 촉진을 위해 해당 제품군을 기본으로 하여 타 제품군 동시 구매 판촉 전략을 짜는 것이 좋아보인다.*

대부분의 제품군 간 연관 규칙 사례가 두 개 이상의 선행 항목(snacks, deli 같이)을 가지고 있는데, deli, canned goods, dry goods pasta, meat seafood, international은 단일 제품군으로 구성되었으면서도 농산물과의 비교적 높은 지지도(구매비율)을 보였습니다. 

이를 볼 때, 해당 제품군들은 타 제품군과의 연계 없이 해당 제품군과 농산물만을 고려한 맞춤형 홍보전략 또한 효과적일 것으로 보입니다.

0.03 이상의 지지도가 나온 규칙들을 분석해봤을 때, dairy eggs와 농산물(produce) 제품군에 속한 것을 다른 제품군과 같이 구매하는 경우가 많았습니다. 

그 중에서도 냉동제품(frozen)과 저장식품(pantry), 조제식품(deli) 계열을 선행 구매할 경우 높은 지지도(0.24, 0.23, 0.20)를 얻었습니다. 

향상도 역시 dairy eggs 등이 후행에 있을 경우 높았습니다. 

반면, 비교한 다른 후행 제품군들은 지지도가 0.03을 넘기지 못하며 그 사례가 드물었습니다. 

이를 볼 때, 재구매활동 촉진을 위해 같이 많이 팔리는 제품군을 기본으로 하여 타 제품군 동시 구매 판촉 전략을 짜거나, 제품 제공에 있어서 효율적인 물류창고 배치를 고려하는 것을 추천할 수 있어 보입니다.